### Actions and Msgs

In [ ]:
def combine_actions_and_msgs(url='/Users/anna/Downloads/ngwl-predict-customer-churn/messages/'):
    import pandas as pd
    import numpy as np

    # открываем данные
    actions = pd.read_csv(url + 'actions.csv')
    messages = pd.read_csv(url + 'messages.csv')

    # переводим в общим формат с actions id
    messages = messages.astype({'action_id': 'int64'})

    # работаем со временем
    messages['sent'] = pd.to_datetime(messages['sent'], unit='s')
    ActM_combine = messages.merge(
        actions, left_on=['action_id'], right_on=['id'])
    ActM_combine['date'] = ActM_combine['sent'].dt.date
    date = pd.to_datetime(ActM_combine['date'])
    ActM_combine['date'] = date
    year = ActM_combine.date.dt.year
    month = ActM_combine.date.dt.month
    day = ActM_combine.date.dt.day
    wday = ActM_combine.date.dt.dayofweek

    # вытаскиваем дни недели и тд
    ActM_combine['date'] = date
    ActM_combine['year'] = year
    ActM_combine['month'] = month
    ActM_combine['day'] = day
    ActM_combine['wday'] = wday

    hours = ActM_combine['sent'].dt.hour
    minutes = ActM_combine['sent'].dt.minute
    ActM_combine['hours'] = hours
    ActM_combine['minutes'] = minutes
    
    # дроп ненужное
    ActM_combine = ActM_combine.drop(['date'], axis=1)
    ActM_combine = ActM_combine.drop(['sent'], axis=1)
    ActM_combine = ActM_combine.drop(['id'], axis=1)
    
    # ЛЭ для эмеил/сообщение/пуш-уведомление
    from sklearn import preprocessing

    type_msg = ActM_combine['type'].to_frame()
    type_msg_le = type_msg.apply(preprocessing.LabelEncoder().fit_transform)
    ActM_combine['type'] = type_msg_le

    return ActM_combine

In [114]:
ActM_combine.head()

,action_id,user_id,subject,body,type,year,month,day,wday,hours,minutes
0,1512,1775692,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,0,2020,5,7,3,10,26
1,1512,1775687,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,0,2020,5,7,3,10,26
2,1512,1775684,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,0,2020,5,7,3,10,26
3,1512,1775678,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,0,2020,5,7,3,10,25
4,1512,1775660,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,0,2020,5,7,3,10,24


### Shipments

In [4]:
line_items01_01 = pd.read_csv('line_items01/line_items_2020-01-01.csv')
line_items01_11 = pd.read_csv('line_items01/line_items_2020-01-11.csv')
line_items01_21 = pd.read_csv('line_items01/line_items_2020-01-21.csv')
line_items01_31 = pd.read_csv('line_items01/line_items_2020-01-31.csv')

In [5]:
line_items01_01.head()

,shipment_id,master_category_id,created_at,price,discount,item_id,quantity,product_id,brand_name,pricer,replaced,cancelled
0,387023,34.0,2018-03-02 17:23:48,64.989998,0.0,3857464,1,14840,Fine Life,Pricer::PerItem,0,0
1,387023,22.0,2018-03-02 17:32:52,22.990000,0.0,3857581,1,66260,Слобода,Pricer::PerItem,0,0
2,387023,22.0,2018-03-02 17:33:24,39.990002,0.0,3857602,1,85595,Epica,Pricer::PerItem,0,0
3,387023,23.0,2018-03-02 17:34:20,59.990002,0.0,3857627,1,12892,Мажитэль,Pricer::PerItem,0,0
4,387023,26.0,2018-03-02 17:34:46,48.900002,0.0,3857685,1,166,Простоквашино,Pricer::PerItem,0,0
